In [0]:
try:
    import gdown
    print("gdown is installed.")
except ImportError:
    print("gdown is not installed. Installing it now...")
    try:
        import subprocess
        subprocess.check_call(["pip", "install", "gdown"])
        print("gdown has been successfully installed.")
        import gdown  # Importing again to use it after installation
    except Exception as e:
        print(f"Error installing gdown: {e}")


from pyspark.ml.feature import VectorAssembler,VectorIndexer, OneHotEncoder,StringIndexer
from pyspark.sql.functions import when, col

import pandas as pd
import numpy as np

try:
    from xgboost.spark import SparkXGBClassifier
    print("sparkxgb is installed.")
except ImportError:
    print("sparkxgb is not installed. Installing it now...")
    try:
        import subprocess
        command = "pip install pyarrow pandas venv-pack xgboost"
        subprocess.run(command, shell=True, check=True)
        print("sparkxgb has been successfully installed.")
        from xgboost.spark import SparkXGBClassifier
    except Exception as e:
        print(f"Error installing sparkxgb: {e}")

from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import FloatType, IntegerType
from pyspark.ml.feature import VectorAssembler,VectorIndexer, OneHotEncoder,StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import os
import glob
import heapq
from pathlib import Path
import shutil




gdown is not installed. Installing it now...
gdown has been successfully installed.


sparkxgb is not installed. Installing it now...
sparkxgb has been successfully installed.


In [0]:
def clear_old_files(folder_path, keep_latest=10):
    # List all files in the folder
    all_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    
    # Sort files by modification time (most recent first)
    all_files.sort(key=lambda f: os.path.getmtime(os.path.join(folder_path, f)), reverse=True)
    
    # Keep the latest files
    files_to_keep = all_files[:keep_latest]
    
    # Delete older files
    for file_name in all_files:
        file_path = os.path.join(folder_path, file_name)
        if file_name not in files_to_keep:
            os.remove(file_path)
            print(f"Deleted: {file_path}")




In [0]:
%%bash
rm -r Trends_market_place_data
# Replace 'YOUR_FILE_ID' with the actual file ID from the Google Drive link
file_id="1SznHDvix3NdpmiIloyDxeAjiFtY-nk38"

# Replace 'output_file.zip' with the desired output file name
output_file="Trends_market_place_data.zip"

# Download the file from Google Drive using gdown
gdown "https://drive.google.com/uc?id=${file_id}" -O "${output_file}"

unzip Trends_market_place_data.zip
rm -f Trends_market_place_data.zip
rm -rf __MACOSX

rm: cannot remove 'Trends_market_place_data': No such file or directory
Downloading...
From (uriginal): https://drive.google.com/uc?id=1SznHDvix3NdpmiIloyDxeAjiFtY-nk38
From (redirected): https://drive.google.com/uc?id=1SznHDvix3NdpmiIloyDxeAjiFtY-nk38&confirm=t&uuid=8e88f336-f0f4-409a-83b9-2a71721ff372
To: /databricks/driver/Trends_market_place_data.zip
100%|██████████| 767M/767M [00:10<00:00, 71.0MB/s]


Archive:  Trends_market_place_data.zip
   creating: Trends_market_place_data/
  inflating: Trends_market_place_data/CTU-IoT-Malware-Capture-35-1conn.log.labeled.csv  
  inflating: __MACOSX/Trends_market_place_data/._CTU-IoT-Malware-Capture-35-1conn.log.labeled.csv  
  inflating: Trends_market_place_data/CTU-IoT-Malware-Capture-3-1conn.log.labeled.csv  
  inflating: __MACOSX/Trends_market_place_data/._CTU-IoT-Malware-Capture-3-1conn.log.labeled.csv  
  inflating: Trends_market_place_data/CTU-IoT-Malware-Capture-9-1conn.log.labeled.csv  
  inflating: __MACOSX/Trends_market_place_data/._CTU-IoT-Malware-Capture-9-1conn.log.labeled.csv  
  inflating: Trends_market_place_data/CTU-IoT-Malware-Capture-1-1conn.log.labeled.csv  
  inflating: __MACOSX/Trends_market_place_data/._CTU-IoT-Malware-Capture-1-1conn.log.labeled.csv  
  inflating: Trends_market_place_data/CTU-IoT-Malware-Capture-21-1conn.log.labeled.csv  
  inflating: __MACOSX/Trends_market_place_data/._CTU-IoT-Malware-Capture-21-1conn.l

In [0]:
%%bash
rm -rf /databricks/driver/tmp 
mkdir /databricks/driver/tmp
mkdir /databricks/driver/tmp/static
rm -rf /databricks/driver/output 
mkdir /databricks/driver/output

In [0]:

#dbutils.fs.rm("dbfs:/databricks/driver/tmp", recurse=True)
#dbutils.fs.mkdirs("dbfs:/databricks/driver/tmp")

#dbutils.fs.rm("dbfs:/databricks/driver/output", recurse=True)
#dbutils.fs.mkdirs("dbfs:/databricks/driver/output")

In [0]:
def process_df(network_labelled_data):
    #Processing data
    columns_to_remove = [0, 2, 4, 12, 13, 14, 20, 22]
    columns_to_drop = [network_labelled_data.columns[i] for i in columns_to_remove]

    # Drop the specified columns
    df_without_columns = network_labelled_data.drop(*columns_to_drop)

    # Define the transformation using when
    df_without_columns = df_without_columns.withColumn(
        "Malicious", 
        when(col("label") == "Benign", 0).otherwise(1)
    )

    # Drop the original label column if needed
    df_without_columns = df_without_columns.drop("label")

    # Columns to transform
    columns_to_transform = [4, 5, 6, 7, 9]
    columns_to_transform = [df_without_columns.columns[i] for i in columns_to_transform]

    # Columns to put 0 if '-'
    zeros = [5, 6, 7]
    zeros = [df_without_columns.columns[i] for i in zeros]

    # Replace '-'
    for column in zeros:
        df_without_columns = df_without_columns.withColumn(
            column,
            when((col(column) == "-") , 0).otherwise(col(column)),
        )

    # Columns to convert to float
    columns_to_convert_to_float = [5]
    columns_to_convert_to_float = [df_without_columns.columns[i] for i in columns_to_convert_to_float]

    for column in columns_to_convert_to_float:
        df_without_columns = df_without_columns.withColumn(column, col(column).cast(FloatType()))

    # Columns to convert to int
    column_mapping_convert = [1, 2]
    column_mapping_convert = [df_without_columns.columns[i] for i in column_mapping_convert]

    # Define the mapping for column renaming
    column_mapping = {
        column_mapping_convert[0]: "id_orig_p",
        column_mapping_convert[1]: "id_resp_p",
    }

    # Rename columns using withColumnRenamed
    for old_col, new_col in column_mapping.items():
        df_without_columns = df_without_columns.withColumnRenamed(old_col, new_col)


    # Columns to convert to int
    columns_to_convert_to_int = [1, 2, 6, 7, 10, 11, 12]
    columns_to_convert_to_int = [df_without_columns.columns[i] for i in columns_to_convert_to_int]


    # Remove rows where the first column contains an IP address
    df_without_columns = df_without_columns.filter(~col(columns_to_convert_to_int[1]).rlike(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"))

    for column in columns_to_convert_to_int:
        df_without_columns = df_without_columns.withColumn(column, col(column).cast(IntegerType()))
    return df_without_columns


In [0]:
! pwd

/databricks/driver


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import random
import os


# Specify the path to the folder containing CSV files
csv_path = r'file:/databricks/driver/Trends_market_place_data/CTU-IoT-Malware-Capture-34-1conn.log.labeled.csv'

# Read the CSV file into a Spark DataFrame
df = spark.read.option("header", "true").option("delimiter", "|").option("inferschema", "true").csv(csv_path)

# Choose a random chunk of data (e.g., 10 rows)
chunk_size = 10
random_chunk = df.sample(False, chunk_size / df.count())
random_chunk = process_df(random_chunk)
# Specify the path to the folder where you want to save the random chunk
output_folder = r'/databricks/driver/tmp/static/'

# Specify the output file path
output_file = output_folder+'random_chunk.csv'

# Save the random chunk to a new CSV file
#random_chunk.limit(1).write.csv(output_file, header=True, mode='overwrite')
random_chunk.limit(1).toPandas().to_csv(output_file, header = True)


In [0]:
random_chunk.limit(1).display()

uid,id_orig_p,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,Malicious
CxRJSN1zDC8dmbgdb3,49708,6667,tcp,-,0.0,0,0,S0,S,1,60,0,0,1


In [0]:
import time
count = 0

# Specify the path to the folder containing CSV files
csv_path = r'file:/databricks/driver/Trends_market_place_data/CTU-IoT-Malware-Capture-34-1conn.log.labeled.csv'
while True:
    count += 1
    # Read the CSV file into a Spark DataFrame
    df = spark.read.option("header", "true").option("delimiter", "|").option("inferschema", "true").csv(csv_path)

    # Choose a random chunk of data (e.g., 10 rows)
    chunk_size = 10
    random_chunk = df.sample(False, chunk_size / df.count())
    random_chunk = process_df(random_chunk)
    # Specify the path to the folder where you want to save the random chunk
    output_folder = r'/databricks/driver/tmp/'

    # Specify the output file path
    output_file = output_folder+'random_chunk_{}.csv'.format(count)

    # Save the random chunk to a new CSV file
    #random_chunk.limit(1).write.csv(output_file, header=True, mode='overwrite')
    random_chunk.limit(1).toPandas().to_csv(output_file, header=True)
    #time.sleep(0.5)
    #clear_old_files(r"/databricks/driver/tmp/", 10)
    print( "round_{}".format(count))
    time.sleep(1)

round_1
round_2
round_3
round_4
round_5
round_6
round_7
round_8
round_9
round_10
round_11
round_12
round_13
round_14
round_15
round_16
round_17
round_18
round_19
round_20
round_21
